In [1]:
import numpy as np
import pandas as pd

In [5]:
def generate_data_with_noise(num_samples = 500, gate = "AND", noise_level = 0.05):

    if gate == 'AND':
      base_X = np.array([[0, 0], [0,1], [1,0], [1,1]])
      base_y = np.array([0,0,0,1])
    elif gate == 'OR':
      base_X = np.array([[0,0], [0,1], [1,0], [1,1]])
      base_y = np.array([0,1,1,1])
    elif gate == 'XOR':
      base_X = np.array([[0,0],[0,1], [1,0], [1,1]])
      base_y = np.array([0,1,1,0])
    else:
      raise ValueError("Gate must be 'And', 'OR', or 'XOR' .")

    X = np.repeat(base_X, num_samples // len(base_X), axis=0)
    y = np.repeat(base_y, num_samples // len(base_y), axis=0)

    X = X + np.random.normal(0, noise_level, X.shape)

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]

    return X,y


In [8]:
from typing_extensions import dataclass_transform
from ipywidgets import interact, FloatSlider, Dropdown, Checkbox
import plotly.express as px
import plotly.graph_objects as go

X, y = generate_data_with_noise(500, 'AND', 0.05)

data_fig = go.FigureWidget()
data_fig.add_trace(go.Scatter(x=X[y == 0,0], y=X[y == 0,1], mode='markers', marker=dict(color='red'), name ='0'))
data_fig.add_trace(go.Scatter(x=X[y == 1, 0], y=X[y == 1,1], mode='markers', marker = dict(color='blue'), name='1'))
data_fig.update_layout(width=800, height=500,
                       xaxis_range=[-1, 2], yaxis_range=[-1, 2])

@interact(num_samples=FloatSlider(min=100, max=1000, step=100, value=500, description='Samples'),
          gate=Dropdown(options=['AND', 'OR', 'XOR'], value='AND', description='Gate'),
          noise_level=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='Noise Level'))
def update_data_plot(num_samples, gate, noise_level):
  X, y = generate_data_with_noise(num_samples,gate,noise_level)
  with data_fig.batch_update():
    data_fig.data[0].x = X[y == 0,0]
    data_fig.data[0].y = X[y == 0,1]
    data_fig.data[1].x = X[y == 1,0]
    data_fig.data[1].y = X[y == 1,1]
    data_fig.update_layout(title=f"Dataset for {gate} Gate with Noise Level {noise_level}")
data_fig

interactive(children=(FloatSlider(value=500.0, description='Samples', max=1000.0, min=100.0, step=100.0), Drop…

FigureWidget({
    'data': [{'marker': {'color': 'red'},
              'mode': 'markers',
              'name': '0',
              'type': 'scatter',
              'uid': 'fe193434-d2b2-4903-90e1-64f6b1f0c9c5',
              'x': array([ 0.94125499,  0.02058708,  0.07224032, ...,  0.01177849, -0.04748487,
                           0.91241537]),
              'y': array([ 0.07635378,  0.90100909, -0.00637636, ..., -0.09404346,  1.04253836,
                           0.02078436])},
             {'marker': {'color': 'blue'},
              'mode': 'markers',
              'name': '1',
              'type': 'scatter',
              'uid': '4838c863-d478-42ec-b002-5391afccbf80',
              'x': array([0.8618953 , 1.04670814, 0.99062129, 1.00838136, 0.98780915, 1.06252714,
                          0.96326927, 0.91324226, 0.92018785, 1.07529705, 0.93849982, 1.04025957,
                          1.0111202 , 0.97532733, 0.97056954, 0.98013353, 0.99553737, 1.01232222,
                       

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def perform_logistic_regression(X,y):
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=140)

  model = LogisticRegression().fit(x_train, y_train)

  train_error = model.score(x_train, y_train)
  test_error = model.score(x_test, y_test)

  return model, train_error, test_error

In [10]:
def plot_decision_boundary(model, xrange, yrange, num_points=100, probs=True):
  xx, yy = np.meshgrid(np.linspace(xrange[0], xrange[1], num_points),
                       np.linspace(yrange[0], yrange[1], num_points))
  grid = np.c_[xx.ravel(), yy.ravel()]
  if probs:
    preds = model.predict_proba(grid)[:,1].reshape(xx.shape)
  else:
    preds = model.predict(grid).reshape(xx.shape)
  return go.Contour(x=xx[0], y=yy[:,0], z=preds, colorscale=[[0,'red'], [1,'blue']],
                    opacity=0.5, showscale=False)


In [12]:
pred_fig = go.FigureWidget(data=data_fig.data, layout=data_fig.layout)

model, train_test, test_error = perform_logistic_regression(X,y)
boundary = plot_decision_boundary(model, [-1, 2], [-1, 2], probs=False)
pred_fig.add_trace(boundary)

@interact(num_samples=FloatSlider(min=100, max=1000, step=100, value=500, description='Samples'),
          gate=Dropdown(options=['AND', 'OR', 'XOR'], value='AND', description='Gate'),
          noise_level=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='Noise Level'),
          show_probs=Checkbox(value=False, description='Show Probabilities'))
def update_pred_fig(num_samples, gate, noise_level, show_probs):
  np.random.seed(42)
  X, y = generate_data_with_noise(num_samples, gate, noise_level)
  model, train_error, test_error = perform_logistic_regression(X, y)
  with pred_fig.batch_update():
    pred_fig.data[0].x = X[y == 0,0]
    pred_fig.data[0].y = X[y == 0,1]
    pred_fig.data[1].x = X[y == 1,0]
    pred_fig.data[1].y = X[y == 1,1]
    pred_fig.data[2].z = plot_decision_boundary(model, [-1, 2], [-1, 2], probs=show_probs).z
    pred_fig.update_layout(title=f"Predictions for {gate} Gate with Noise Level {noise_level} (Train: {train_error:.2f}, Test: {test_error:.2f})")

pred_fig

interactive(children=(FloatSlider(value=500.0, description='Samples', max=1000.0, min=100.0, step=100.0), Drop…

FigureWidget({
    'data': [{'marker': {'color': 'red'},
              'mode': 'markers',
              'name': '0',
              'type': 'scatter',
              'uid': 'ad2a7be8-c7a0-48a6-a856-ea32f8b06b88',
              'x': array([ 0.97386385, -0.03232864, -0.03008533, ...,  0.98181939,  0.91106399,
                           0.96942411]),
              'y': array([ 0.05245046,  0.9459226 ,  0.09261391, ..., -0.00284728,  0.07480222,
                          -0.07033305])},
             {'marker': {'color': 'blue'},
              'mode': 'markers',
              'name': '1',
              'type': 'scatter',
              'uid': 'd60a6121-b1bc-4bcc-b917-f0b2bd3c8fd3',
              'x': array([0.96364314, 0.97224002, 0.94893836, 1.0246659 , 0.97411944, 1.03409457,
                          1.02798952, 0.93598478, 0.97767832, 1.01445843, 0.99065642, 1.00141592,
                          1.01335251, 1.00955495, 1.04691419, 1.07737526, 0.96936057, 0.96865165,
                       

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

In [14]:
def make_tensors(X,y):
  from torch.utils.data import random_split, TensorDataset
  data = TensorDataset(torch.tensor(X, dtype=torch.float32),
                       torch.tensor(y, dtype=torch.float32).unsqueeze(1))
  torch.manual_seed(140)
  train_data, test_data = random_split(data, [0.8,0.2])
  return train_data, test_data

In [21]:
class LogisticRegressionModelA(nn.Module):
  def __init__(self, input_size):
    super().__init__()
    self.intercept = nn.Parameter(torch.tensor(1.0))
    self.w = nn.Parameter(torch.ones(input_size, 1))

  def forward(self, x):
    intercept = self.intercept
    w = self.w
    z = intercept + torch.matmul(x, w)
    return torch.sigmoid(z)

In [16]:
class LogisticRegressionModelB(nn.Module):
  def __init__(self, input_size):
    super().__init__()
    self.linear = nn.Linear(input_size, 1)

  def forward(self, x):
    return torch.sigmoid(self.linear(x))

In [17]:
model = LogisticRegressionModelB(2)

In [18]:
loss_fn = nn.BCELoss()

In [23]:
def perform_logistic_regression_pytorch(train_dataset,
                                        test_dataset,
                                        model, loss_fn,
                                        batch_size=64,
                                        nepochs=20):

  from torch.utils.data import DataLoader

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  optimizer = optim.SGD(model.parameters(), lr=0.5)

  for epoch in range(nepochs):
    for batch, (X, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pred = model(X)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()

    with torch.no_grad():
      test_loss_sum = 0.0
      for X_test, y_test in test_loader:
        test_pred = model(X_test)
        test_loss = loss_fn(test_pred, y_test)
        test_loss_sum += test_loss.item()
      num_test_batches = len(test_loader)
      print(f"Epoch {epoch}, Loss: {loss.item()}, Test Loss: {test_loss_sum/num_test_batches}")

In [24]:
train_dataset, test_dataset = make_tensors(X, y)
model = LogisticRegressionModelA(2)
perform_logistic_regression_pytorch(train_dataset, test_dataset, model, loss_fn)

Epoch 0, Loss: 0.5961155295372009, Test Loss: 0.5737093985080719
Epoch 1, Loss: 0.4994142949581146, Test Loss: 0.4806044101715088
Epoch 2, Loss: 0.32435765862464905, Test Loss: 0.4411657005548477
Epoch 3, Loss: 0.3053455054759979, Test Loss: 0.4076780080795288
Epoch 4, Loss: 0.35575464367866516, Test Loss: 0.3830236494541168
Epoch 5, Loss: 0.306686669588089, Test Loss: 0.3565913587808609
Epoch 6, Loss: 0.3297501504421234, Test Loss: 0.33291158080101013
Epoch 7, Loss: 0.25527986884117126, Test Loss: 0.31530721485614777
Epoch 8, Loss: 0.29264402389526367, Test Loss: 0.3004146069288254
Epoch 9, Loss: 0.32083186507225037, Test Loss: 0.28473563492298126
Epoch 10, Loss: 0.22501149773597717, Test Loss: 0.27071061730384827
Epoch 11, Loss: 0.2106669843196869, Test Loss: 0.261477455496788
Epoch 12, Loss: 0.2554934024810791, Test Loss: 0.24979925155639648
Epoch 13, Loss: 0.27667832374572754, Test Loss: 0.23814814537763596
Epoch 14, Loss: 0.2517881691455841, Test Loss: 0.22931134700775146
Epoch 15

In [25]:
def plot_decision_boundary_pytorch(model, xrange, yrange, num_points=100, probs=True):
  xx, yy = torch.meshgrid(torch.linspace(xrange[0], xrange[1], num_points),
                          torch.linspace(yrange[0], yrange[1], num_points),
                          indexing='ij')
  grid = torch.cat([xx.reshape(-1,1), yy.reshape(-1,1)], dim=1)
  with torch.no_grad():
    if probs:
      preds = model(grid).reshape(xx.shape)
    else:
      preds = (model(grid) > 0.5).float().reshape(xx.shape)
  return go.Contour(x=xx[:,0], y=yy[0], z=preds, colorscale=[[0,'red'], [1, 'blue']],
                    opacity=0.5, showscale=False)


In [27]:
pred_fig = go.FigureWidget(data=data_fig.data, layout=data_fig.layout)
model_type = LogisticRegressionModelA

train_dataset, test_dataset = make_tensors(X,y)
model = model_type(2)
perform_logistic_regression_pytorch(train_dataset, test_dataset, model, loss_fn)
boundary = plot_decision_boundary_pytorch(model, [-1,2], [-1,2], probs=False)
pred_fig.add_trace(boundary)

display(pred_fig)

@interact(num_samples=FloatSlider(min=100,max=1000,step=100,value=500, description='Samples'),
          gate=Dropdown(options=['AND', 'OR', 'XOR'], value='AND', description='Gate'),
          noise_level=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='Noise Level'),
          show_probs=Checkbox(value=False, description='Show Probabilities'))

def update_pred_fig(num_samples, gate, noise_level, show_probs):
  np.random.seed(42)
  X, y = generate_data_with_noise(num_samples, gate, noise_level)
  train_dataset, test_dataset = make_tensors(X, y)
  model = model_type(2)
  perform_logistic_regression_pytorch(train_dataset, test_dataset, model, loss_fn)
  boundary = plot_decision_boundary_pytorch(model, [-1, 2], [-1, 2], probs=show_probs)
  with pred_fig.batch_update():
    pred_fig.data[0].x = X[y == 0,0]
    pred_fig.data[0].y = X[y == 0,1]
    pred_fig.data[1].x = X[y == 1,0]
    pred_fig.data[1].y = X[y == 1,1]
    pred_fig.data[2].z = boundary.z

Epoch 0, Loss: 0.5961155295372009, Test Loss: 0.5737093985080719
Epoch 1, Loss: 0.4994142949581146, Test Loss: 0.4806044101715088
Epoch 2, Loss: 0.32435765862464905, Test Loss: 0.4411657005548477
Epoch 3, Loss: 0.3053455054759979, Test Loss: 0.4076780080795288
Epoch 4, Loss: 0.35575464367866516, Test Loss: 0.3830236494541168
Epoch 5, Loss: 0.306686669588089, Test Loss: 0.3565913587808609
Epoch 6, Loss: 0.3297501504421234, Test Loss: 0.33291158080101013
Epoch 7, Loss: 0.25527986884117126, Test Loss: 0.31530721485614777
Epoch 8, Loss: 0.29264402389526367, Test Loss: 0.3004146069288254
Epoch 9, Loss: 0.32083186507225037, Test Loss: 0.28473563492298126
Epoch 10, Loss: 0.22501149773597717, Test Loss: 0.27071061730384827
Epoch 11, Loss: 0.2106669843196869, Test Loss: 0.261477455496788
Epoch 12, Loss: 0.2554934024810791, Test Loss: 0.24979925155639648
Epoch 13, Loss: 0.27667832374572754, Test Loss: 0.23814814537763596
Epoch 14, Loss: 0.2517881691455841, Test Loss: 0.22931134700775146
Epoch 15

FigureWidget({
    'data': [{'marker': {'color': 'red'},
              'mode': 'markers',
              'name': '0',
              'type': 'scatter',
              'uid': 'cf075fd1-80a8-41a7-a0fa-a903c41b0a47',
              'x': array([-0.14141304,  0.45258771, -0.07709595, -0.57317836, -0.65310265,
                          -0.56173555, -0.15065865,  0.42664052,  0.74836273,  0.16517107,
                           0.36954586, -0.68867428, -1.11322198,  0.44202095,  0.22942262,
                          -0.1249467 ,  0.78798655, -0.04252901,  0.25843461, -0.40027174,
                          -0.61192152, -0.06450327, -0.1835859 ,  0.51320087, -0.39085904,
                           0.2844211 ,  0.00173133, -1.04830258,  0.01496851, -0.07670513,
                           0.66649399,  0.39678092,  0.54465069,  0.02907248,  0.68910639,
                           0.09698482,  0.57041695, -0.31737919,  0.06560465, -0.3877403 ,
                           0.46195671, -0.4767655 , -0.279203

interactive(children=(FloatSlider(value=500.0, description='Samples', max=1000.0, min=100.0, step=100.0), Drop…